In [1]:
from  transformers import BertTokenizer, BertForTokenClassification
import torch

c:\Users\Jorge\anaconda3\envs\Extract\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
etiquetas = {
    'O' : 0,
    "FECHA" : 1,
    "DIRECTOR" : 2,
    "ROL_ENTITY": 3,
    "EDICION" : 4,
    "TIPO_DOCUMENTO" : 5,
    "ROMAN_NUM" : 6,
    "NUM_ISSN" : 7,
    "ENTITY": 8,
    "PRESENTATION": 9
}

# load model and tokenizer

tokenizer = BertTokenizer.from_pretrained('dccuchile/bert-base-spanish-wwm-cased', num_labels=len(etiquetas))
model = BertForTokenClassification.from_pretrained('dccuchile/bert-base-spanish-wwm-cased', num_labels=len(etiquetas))

# Move the model to the GPU 
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


Some weights of BertForTokenClassification were not initialized from the model checkpoint at dccuchile/bert-base-spanish-wwm-cased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


BertForTokenClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(31002, 768, padding_idx=1)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12

## TOKENIZAR AND CODIFY THE DATA

In [3]:
def tokenizar_codify(phrase, tokenizer, tags):
    tokens_codified = tokenizer(
        [token for token, tag in phrase],
        is_split_into_words=True,  # True if the input is already tokenized
        return_tensors="pt",
        padding=True,
        truncation=True,
    )
    tags_codified = torch.tensor([tags[tag] for token, tag in phrase])
    return tokens_codified, tags_codified

In [4]:
data_train = [
    [("Bogotá", "O"), ("D.C.", "O"), (",", "O"), ("jueves", "O"), (",", "O"), ("13", "FECHA"), ("de", "O"), ("diciembre", "FECHA"), ("de", "O"), ("2019", "FECHA")],
    [("GREGORIO", "DIRECTOR"), ("ELJACH", "DIRECTOR"), ("PACHECO", "DIRECTOR"), ("Y", "O"), ("MARIA", "DIRECTOR"), ("GOMEZ", "DIRECTOR"), (".", "O")],
    [("DIRECORES", "DIRECTOR"), ("SECRETARIO", "ROL_ENTITY"), ("GENERAL", "ROL_ENTITY"), ("DE", "O"), ("LA", "O"), ("CÁMARA", "ROL_ENTITY")],
    [("GREGORIO", "DIRECTOR"), ("ELJACH", "DIRECTOR"), ("PACHECO", "DIRECTOR"), ("Y", "O"), ("MARIA", "DIRECTOR"), ("GOMEZ", "DIRECTOR"), (".", "O")],
    [("DIRECORES", "DIRECTOR"), ("SECRETARIO", "ROL_ENTITY"), ("GENERAL", "ROL_ENTITY"), ("DEL", "O"), ("SENADO", "ROL_ENTITY")]
]

In [5]:
tokens_codified, tags_codified = tokenizar_codify(data_train[0], tokenizer, etiquetas)
print(tokens_codified)
print(tags_codified)

{'input_ids': tensor([[    4, 17521,  1090,  1009,  1059,  1009,  1017, 11573,  1017,  2083,
          1008,  2451,  1008, 17736,     5]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
tensor([0, 0, 0, 0, 0, 1, 0, 1, 0, 1])


Este resultado muestra la arquitectura del modelo BERT que se ha cargado, incluyendo las capas de embeddings, atención, y clasificación. La advertencia inicial indica que algunas de las capas del clasificador (classifier.bias y classifier.weight) no se han inicializado desde el punto de control del modelo y se han inicializado de nuevo. Esto es normal cuando se carga un modelo preentrenado y se ajusta para una tarea específica.

